# Pairs Trading 2

Leo Luo

Deeper Analysis of Pairs Trading

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import Quandl
import functools
import seaborn as sns
import ggplot as gg

## Declaration of Functions

In [3]:
#save data to cache
@functools.lru_cache(maxsize=16)
def fetch_quandl(my_securities):
    qdata = Quandl.get(list(my_securities), trim_start='2013-12-02', trim_end='2015-12-31', returns="pandas", authtoken="TzTcmaHpPQtRD3s_g2Z3")
    return qdata

#clean and make data look nicer
def clean_quandl_columns(dataframe):
    replacement_columns = {}
    for c in dataframe.columns:
        series_name, variable = c.split(' - ')
        source_name, asset = series_name.split('.')
        replacement_columns[c] = asset+":"+variable
    renamed_data = dataframe.rename(columns=replacement_columns)
    return renamed_data


#renamed_data = clean_quandl_columns(raw_data)
#renamed_data.tail()

In [4]:
#preStratClean
def preStratClean(raw_data, num):
    renamed_data = clean_quandl_columns(raw_data)
    cleaned = renamed_data[[SymList.ix[num,'x']+":Adj_Volume",SymList.ix[num,'x']+":Adj_Close",SymList.ix[num,'y']+":Adj_Close"]].copy()
    cleaned.columns = ["A:Adj_Volume", "A:Adj_Close", "B:Adj_Close"]
    cleaned.ix[:,'Nt'] = np.nan
    #Get the 15-day median of rolling volume as required
    DollarVolume = cleaned['A:Adj_Close']  * cleaned['A:Adj_Volume']
    N = 15
    Ntdata = DollarVolume.rolling(center=False,window =N).median()
    cleaned.ix[N-1:,'Nt'] = Ntdata
    cleaned['month'] = cleaned.index.month
    #Derive a column that indicate end of month
    cleaned['endofmonth'] = cleaned.month-cleaned.month.shift(-1)
    cleaned.ix[-1,'endofmonth']=0
    cleaned.ix[abs(cleaned['endofmonth'])>0, 'endofmonth'] =1
    return cleaned

In [5]:
#STRATEGY THAT RETURN DAILY p&l AND TOTAL_PNL
def run_strat(df2, M, g, j, s):
    #or we could use simple
    #df2['A_Mret'] = df2['A:Adj_Close'].pct_change(periods = M-1)
    #df2['B_Mret'] = df2['B:Adj_Close'].pct_change(periods = M-1)
    tb = df2.copy()
    tb['A_Mret'] = tb['A:Adj_Close'].rolling(M).apply(lambda x: np.log(x[-1]/x[0]))
    tb['B_Mret'] = tb['B:Adj_Close'].rolling(M).apply(lambda x: np.log(x[-1]/x[0]))
    tb['Diff_Mret'] = tb['A_Mret'] - tb['B_Mret'] #diff or Mret
    tb['Stoplose'] =0 #stoploss indicator
    tradingday = max(M, 15)
    
    preidx = tb.index[tradingday-1]
    #testdata = tb[tradingday:];
    entry_gtc = 0 #entry gross traded capital
    A_Pos_i = 0 # Position of stock A in time i
    B_Pos_i = 0
    daypnl =[]
    cur_ret = 0
    for idx_t, eom in zip(tb[tradingday:].index, tb[tradingday:].endofmonth):
        dollar_size = tb.ix[idx_t, 'Nt'] / 100 #* tb.ix[i, 'A:Adj_Close']
        dayret = (tb.ix[idx_t, 'A:Adj_Close'] - tb.ix[preidx, 'A:Adj_Close']) * A_Pos_i  + \
                (tb.ix[idx_t, 'B:Adj_Close'] - tb.ix[preidx, 'B:Adj_Close']) * B_Pos_i
        daypnl.append(dayret)
        cur_ret += dayret
        #if end of month, close position
        if(abs(eom)> 0.1):
            A_Pos_i = 0
            B_Pos_i = 0
            entry_gtc = 0
            cur_ret = 0
        #if satisfy stoploss, then stoploss
        elif(np.logical_and(abs(entry_gtc)>1, cur_ret/entry_gtc < -s)):
            A_Pos_i = 0
            B_Pos_i = 0
            entry_gtc = 0
            cur_ret = 0
        #if satisfy buy or sell, then get position
        elif(abs(A_Pos_i)<1 and (abs(tb.ix[idx_t, 'Diff_Mret']) > g).any()):
            A_Pos_i = - np.sign(abs(tb.ix[idx_t, 'Diff_Mret']))* int(round(dollar_size / tb.ix[idx_t, 'A:Adj_Close'])) 
            B_Pos_i = + np.sign(abs(tb.ix[idx_t, 'Diff_Mret']))* int(round(dollar_size / tb.ix[idx_t, 'B:Adj_Close'])) 
            entry_gtc = abs(A_Pos_i * tb.ix[i, 'A:Adj_Close']) + \
                abs(B_Pos_i * tb.ix[i, 'B:Adj_Close'])
        #if satisfy close position, then close
        elif ((abs(tb.ix[idx_t, 'Diff_Mret']) < j).any() or (tb.ix[idx_t, 'Diff_Mret']* tb.ix[preidx, 'Diff_Mret']) < 0):
            A_Pos_i = 0
            B_Pos_i = 0
            entry_gtc = 0
            cur_ret = 0
        preidx = idx_t
    pnl = np.cumsum(daypnl)
    daypnl = daypnl/ tb['Nt'].max()
    pnl = pnl/tb['Nt'].max()
    pnl_final = sum(daypnl)
    return (daypnl, pnl, pnl_final)

In [19]:
#SEARCH FOR RELIABLE G, J
def findGF(df2, M,g,j,s):
    column_names = ['M','g','j','s','pnl'];
    result = pd.DataFrame(columns=column_names)
    for i in range(0, len(g)):
        for k in range(0, len(j)):
            if g[i]<j[k]:
                pnl_final = 0.00
            else:
                (_, _, pnl_final) = run_strat(df2, M, g[i], j[k], s)
            #result[j][k] = pnl_final
            resrow = pd.DataFrame([[M, g[i], j[k], s,pnl_final]],columns=column_names )
            result = result.append(resrow ,ignore_index=True)
    rs = result.pivot("g","j","pnl")
    bestg = max(rs.idxmax())    
    bestj = rs.max()[rs.max() == rs.max(axis=1).max()].index[0]
    (_, _, pnl_final_best) = run_strat(df2, M, bestg, bestj, s)  
    return bestg, bestj,pnl_final_best



In [7]:
Symbols = [['RYU','XLU'],['IST','IYZ'],['RING','GDX'],['XSD','SMH'],['PBE','XBI'],['IEO','XOP'],['PXJ','OIH'],['RTH','XRT'],['SIVR','SLV'],['HYLD','JNK']]
SymList = pd.DataFrame(Symbols,  index =range(10), columns = ['x','y'])

## Find Reliable g and f

In [36]:
# FIND THE RELIABLE g AND F FOR ALL PAIRS
# I have chosen 11*11 pairs of g and f and I will choose the best final P&L pair as our result 
M=20
g = np.linspace(0.003,0.0503,num=11)
j = np.linspace(0.0001,0.0101, num = 11)
s = 1.00
GFList = np.zeros((10,3))
for i in range(10):
    Xstr = "EOD/" + SymList.ix[i,'x']
    Ystr = "EOD/" + SymList.ix[i,'y']
    raw_data = fetch_quandl((Xstr, Ystr))
    df2 = preStratClean(raw_data,i)
    bestg, bestj, pnl_final_best =  findGF(df2, M,g,j,s)
    GFList[i,0],GFList[i,1],GFList[i,2]  = bestg, bestj, pnl_final_best
GF  =  pd.DataFrame(GFList,  index =range(10), columns = ['bestg','bestf','pnl_final_best'])
    

In [37]:
# show the best g and j under my selections
GF.T
#pnl down here is the percentage pnl

,0,1,2,3,4,5,6,7,8,9
bestg,0.026650,0.040840,0.026650,0.040840,0.026650,0.040840,0.012460,0.03138,0.003000,0.012460
bestf,0.000100,0.000100,0.000100,0.005100,0.003100,0.004100,0.003100,0.00110,0.000100,0.001100
pnl_final_best,0.000077,0.000178,0.000018,0.000343,0.000519,0.000176,0.000228,0.00005,0.000033,0.000997


Now we get the reasonable g, f, and we could continue to next step.


## Get all PNL

In [50]:
#get all pnl
M = 20
#pnlList = pnl
raw_data = fetch_quandl("EOD/" + SymList.ix[i,'x'])
dateind = df2.index[M:]
pnlList = pd.DataFrame(index = dateind)

for i in range(10):
    Xstr = "EOD/" + SymList.ix[i,'x']
    Ystr = "EOD/" + SymList.ix[i,'y']
    g = GF.ix[i,'bestg']
    j = GF.ix[i,'bestf']
    raw_data = fetch_quandl((Xstr, Ystr))
    df2 = preStratClean(raw_data,i)
    daypnl, _ , _ = run_strat(df2, M,g , j, s)
    pnlList['Pair'+ str(i)] = daypnl

In [126]:
#fama_french data
M = 20
famaStat = Quandl.get("KFRENCH/FACTORS_D", trim_start='2013-12-02', trim_end='2015-12-31', returns="pandas", authtoken="TzTcmaHpPQtRD3s_g2Z3")
famaStat = famaStat[M:] #SINCE p&l START FROM 20
rfRate = famaStat['RF']

In [127]:
famaStat.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
2013-12-31,0.44,-0.19,0.09,0.0
2014-01-02,-0.88,-0.24,0.11,0.0
2014-01-03,0.03,0.37,0.03,0.0
2014-01-06,-0.34,-0.56,0.26,0.0
2014-01-07,0.68,0.37,-0.39,0.0


## Calculate SS from only the pnl

In [128]:
def calSS(dailypnl, rfRate):
    SharpeR = 10000 * (dailypnl - rfRate).mean() / (10000*dailypnl - 10000*rfRate).std()  #since std is so small, so I time 10000 here
    negdailypnl = dailypnl[dailypnl<0].copy()
    SortinoR = (dailypnl - rfRate).mean() / negdailypnl.std()
    return SharpeR, SortinoR

In [129]:
PNLSSMatrix = pd.DataFrame(index =['Sharpe','Sortino'], columns = range(10))
for i in range(10):
    PNLSSMatrix.ix['Sharpe',i], PNLSSMatrix.ix['Sortino',i] = calSS(pnlList['Pair' + str(i)], rfRate)
PNLSSMatrix    

,0,1,2,3,4,5,6,7,8,9
Sharpe,0.0344526,0.0167887,0.00617469,0.041262,0.0425584,0.0189797,0.0498674,0.0073196,0.0230329,0.114311
Sortino,0.0123108,0.0169834,0.00138014,0.0314871,0.0535852,0.0168234,0.0507189,0.00522235,0.00880291,0.0999997


Above is the Sharpe Ratio and Sortino Ratio of different pairs using my strategy

## Running Regression on Single Factors.

In [130]:
import statsmodels.formula.api as smf
RegData = pd.merge(pnlList, famaStat, left_index =True, right_index = True)
idxname =[]
for famaF in famaStat:
    idxname.append(famaF + 'Sharpe')
    idxname.append(famaF + 'Sortino')
SSList_singleF = pd.DataFrame(index =idxname, columns = range(10))

for famaF in famaStat:
    for i in range(10):
        spread_strat = pnlList['Pair'+str(i)]
        benchmark = famaStat[famaF]
        mod = smf.ols(formula='spread_strat~benchmark+0', data = RegData)
        res = mod.fit()
        SSList_singleF.ix[famaF + 'Sharpe' ,i], SSList_singleF.ix[famaF + 'Sortino',i] = calSS(res.resid,rfRate)
        


In [131]:
SSList_singleF

,0,1,2,3,4,5,6,7,8,9
Mkt-RFSharpe,0.0343197,0.01355,0.00597458,0.0455797,0.0385826,0.0171131,0.0516323,0.0025808,0.0223941,0.116221
Mkt-RFSortino,0.0615511,0.0181902,0.00552077,0.0545832,0.0557459,0.0218281,0.0592747,0.0031234,0.0272679,0.158413
SMBSharpe,0.0346398,0.0229254,0.00927619,0.0341836,0.0603929,0.027354,0.0435128,0.0172223,0.0209712,0.110593
SMBSortino,0.0584977,0.0300976,0.00821308,0.0448034,0.0933354,0.0347642,0.0521051,0.0198647,0.0249997,0.162791
HMLSharpe,0.0346298,0.0109852,0.0123778,0.0444545,0.0249826,0.0277989,0.047614,0.00804751,0.0266231,0.108367
HMLSortino,0.0560547,0.014637,0.0106066,0.051191,0.0374385,0.0342889,0.0560296,0.0086632,0.0291586,0.161875
RFSharpe,0.0344526,0.0167887,0.00617469,0.041262,0.0425584,0.0189797,0.0498674,0.0073196,0.0230329,0.114311
RFSortino,0.0123108,0.0169834,0.00138014,0.0314871,0.0535852,0.0168234,0.0507189,0.00522235,0.00880291,0.0999997


In [132]:
# now we try to see if the hedged ones have better sharpe or Sortino, therefore we are going to compare it. 
#original one: PNLSSMatrix    
ComparisonMatrix = SSList_singleF
#ComparisonMatrix.ix[famaF + 'Sharpe'] 
for famaF in famaStat:
    ComparisonMatrix.ix[famaF + 'Sharpe'] = ComparisonMatrix.ix[famaF + 'Sharpe'] > PNLSSMatrix.ix['Sharpe']
    ComparisonMatrix.ix[famaF + 'Sortino'] = ComparisonMatrix.ix[famaF + 'Sortino'] > PNLSSMatrix.ix['Sortino']
ComparisonMatrix

,0,1,2,3,4,5,6,7,8,9
Mkt-RFSharpe,False,False,False,True,False,False,True,False,False,True
Mkt-RFSortino,True,True,True,True,True,True,True,False,True,True
SMBSharpe,True,True,True,False,True,True,False,True,False,False
SMBSortino,True,True,True,True,True,True,True,True,True,True
HMLSharpe,True,False,True,True,False,True,False,True,True,False
HMLSortino,True,False,True,True,False,True,True,True,True,True
RFSharpe,False,False,False,False,False,False,False,False,False,False
RFSortino,False,False,False,False,False,False,False,False,False,False


The Above chart shows if using Fama Factor adjusted return are good or bad.
If it shows "True", it means the adjusted one is better than the original one. 

In [133]:
#counting number of true in the above chart, 10 means all are "true"
ComparisonMatrix.sum(1)

Mkt-RFSharpe      3
Mkt-RFSortino     9
SMBSharpe         6
SMBSortino       10
HMLSharpe         6
HMLSortino        8
RFSharpe          0
RFSortino         0
dtype: int64

We can see that except the RF adjusted, all other adjusted will likely to create a better Sortino Ratio. For Sharpe Ratio, the change is unpredictable.

## Regression on all factors

In [134]:
import statsmodels.formula.api as smf
SSList_4F = pd.DataFrame(index =['4factor_Sharpe', '4factor_Sortino'], columns = range(10))
#SINCE THE - SIGN IS NOT WORKING IN RUNNING REGRESSION, I CHANGED IT TO _
famaStat = famaStat.rename(columns ={'Mkt-RF':'MktRF'})
RegData = pd.merge(pnlList, famaStat, left_index =True, right_index = True)

for i in range(10):
    spread_strat = pnlList['Pair'+str(i)]
    mod = smf.ols(formula='spread_strat~ MktRF+SMB+HML+RF+0', data = RegData)
    res = mod.fit()
    (SSList_4F.ix['4factor_Sharpe' ,i], SSList_4F.ix['4factor_Sortino',i]) = calSS(res.resid,rfRate)
SSList_4F

,0,1,2,3,4,5,6,7,8,9
4factor_Sharpe,0.0348887,0.016678,0.0193247,0.0365297,0.0421062,0.0436134,0.0373842,0.0211168,0.0236948,0.101915
4factor_Sortino,0.0577675,0.0227274,0.0164688,0.0483259,0.0648547,0.0539978,0.0464668,0.0244348,0.027495,0.159894


In [137]:
ComparisonMatrix2 = SSList_4F.copy()
ComparisonMatrix2.ix['4factor_Sharpe']
for famaF in famaStat:
    ComparisonMatrix2.ix['4factor_Sharpe'] = ComparisonMatrix2.ix['4factor_Sharpe'] > PNLSSMatrix.ix['Sharpe']
    ComparisonMatrix2.ix['4factor_Sortino'] = ComparisonMatrix2.ix['4factor_Sortino'] > PNLSSMatrix.ix['Sortino']
ComparisonMatrix2

,0,1,2,3,4,5,6,7,8,9
4factor_Sharpe,True,False,True,False,False,True,False,True,True,False
4factor_Sortino,True,True,True,True,True,True,False,True,True,True


Still, Sortino ratio are improved, but sharpe ratio changes unpredictable